In [11]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, f1_score, roc_auc_score
import joblib
import os
import glob
import zipfile
import json

# JSON 파일 불러오기 함수
def load_json_data(folder_path, label_as_harmful=True):
    all_files = glob.glob(os.path.join(folder_path, "**", "*.json"), recursive=True)
    if not all_files:
        print(f"경로 '{folder_path}'에서 JSON 파일을 찾을 수 없습니다.")
        return pd.DataFrame()
    
    data_list = []
    print(f"✅ '{folder_path}'에서 JSON 파일을 읽고 있습니다. (총 {len(all_files)}개)")
    
    # '말뭉치' 데이터는 'corpus' 키를, '유해질의' 데이터는 'instruct_text' 키를 사용
    if 'normal' in folder_path:
        text_key = 'corpus'
    else:
        text_key = 'instruct_text'
        
    for file_path in all_files:
        try:
            with open(file_path, 'r', encoding='utf-8-sig') as f:
                json_data = json.load(f)
                
                # 'data' 키가 있는지 확인하고, 있으면 그 안에서 텍스트를 추출
                if 'data' in json_data and isinstance(json_data['data'], list):
                    for item in json_data['data']:
                        text = item.get(text_key, '')
                        label = 1 if label_as_harmful else 0
                        data_list.append({'text': text, 'label': label})
                # 'data' 키가 없으면 JSON 파일 자체가 항목이라고 가정
                elif isinstance(json_data, dict):
                    text = json_data.get(text_key, '')
                    label = 1 if label_as_harmful else 0
                    data_list.append({'text': text, 'label': label})
                else:
                    print(f"Error: {file_path} 파일의 예상치 못한 구조: {type(json_data)}")
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            continue

    df = pd.DataFrame(data_list)
    return df

# 압축 해제 함수 (유해/정상 데이터 구분)
def unzip_data(zip_folder_path, extract_path, zip_file_pattern):
    os.makedirs(extract_path, exist_ok=True)
    zip_files = glob.glob(os.path.join(zip_folder_path, zip_file_pattern))
    
    if not zip_files:
        print(f"Error: '{zip_folder_path}' 경로에서 '{zip_file_pattern}' 파일을 찾을 수 없습니다. 경로를 다시 확인해주세요.")
        return False
        
    print(f"✅ '{zip_file_pattern}' ZIP 파일 압축을 해제합니다.")
    for zip_file in zip_files:
        with zipfile.ZipFile(zip_file, 'r') as zf:
            zf.extractall(extract_path)
    print("✅ 압축 해제 완료!")
    return True

# ZIP 파일이 있는 원본 경로
training_zip_path = r'C:\Users\admin\Downloads\119.국가기록물 대상 초거대AI 학습을 위한 말뭉치 데이터\3.개방데이터\1.데이터\Training'
validation_zip_path = r'C:\Users\admin\Downloads\119.국가기록물 대상 초거대AI 학습을 위한 말뭉치 데이터\3.개방데이터\1.데이터\Validation'

# 압축을 해제할 새로운 경로
extracted_harmful_training = r'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_harmful_data_training'
extracted_harmful_validation = r'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_harmful_data_validation'
extracted_normal_training = r'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_normal_data_training'
extracted_normal_validation = r'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_normal_data_validation'

try:
    # 1. 유해 데이터 압축 해제
    unzip_harmful_train_ok = unzip_data(os.path.join(training_zip_path, '02.라벨링데이터'), extracted_harmful_training, '*유해질의*.zip')
    unzip_harmful_val_ok = unzip_data(os.path.join(validation_zip_path, '02.라벨링데이터'), extracted_harmful_validation, '*유해질의*.zip')
    # 2. 정상(말뭉치) 데이터 압축 해제
    unzip_normal_train_ok = unzip_data(os.path.join(training_zip_path, '01.원천데이터'), extracted_normal_training, '*말뭉치*.zip')
    unzip_normal_val_ok = unzip_data(os.path.join(validation_zip_path, '01.원천데이터'), extracted_normal_validation, '*말뭉치*.zip')

    if not all([unzip_harmful_train_ok, unzip_harmful_val_ok, unzip_normal_train_ok, unzip_normal_val_ok]):
        raise ValueError("모든 데이터 압축 해제 실패. 경로를 다시 확인해주세요.")

    # 3. 데이터 로드 및 라벨링
    harmful_train_df = load_json_data(extracted_harmful_training, label_as_harmful=True)
    harmful_val_df = load_json_data(extracted_harmful_validation, label_as_harmful=True)
    normal_train_df = load_json_data(extracted_normal_training, label_as_harmful=False)
    normal_val_df = load_json_data(extracted_normal_validation, label_as_harmful=False)

    if any([df.empty for df in [harmful_train_df, harmful_val_df, normal_train_df, normal_val_df]]):
        raise ValueError("데이터 로드 실패: JSON 파일을 읽지 못했습니다.")

    print("\n데이터를 성공적으로 불러왔습니다.")
    
    # 4. 모든 데이터 합치기
    df = pd.concat([harmful_train_df, harmful_val_df, normal_train_df, normal_val_df], ignore_index=True)
    
    # 중복 제거 및 텍스트 전처리
    df.drop_duplicates(subset=['text'], inplace=True)
    df.dropna(subset=['text'], inplace=True)
    df = df[df['text'].str.strip() != '']
    print("전체 데이터셋 크기:", df.shape)
    
    def clean_text(text):
        text = str(text)
        text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', text)
        text = re.sub(r'[^\w\s\n]|<[^>]+>',' ', text).strip()
        return ' '.join(text.split())

    df['text'] = df['text'].apply(clean_text)
    
    # 5. 데이터 분할 및 모델 학습
    X_train, X_temp, y_train, y_temp = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42, stratify=df['label'])
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
    tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    
    model = LogisticRegression(class_weight='balanced')
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    print("\n--- 유해성 메일 분류 모델 성능 ---")
    print(classification_report(y_test, y_pred))

except (FileNotFoundError, ValueError) as e:
    print(f"Error: {e}")

✅ '*유해질의*.zip' ZIP 파일 압축을 해제합니다.
✅ 압축 해제 완료!
✅ '*유해질의*.zip' ZIP 파일 압축을 해제합니다.
✅ 압축 해제 완료!
✅ '*말뭉치*.zip' ZIP 파일 압축을 해제합니다.
✅ 압축 해제 완료!
✅ '*말뭉치*.zip' ZIP 파일 압축을 해제합니다.
✅ 압축 해제 완료!
✅ 'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_harmful_data_training'에서 JSON 파일을 읽고 있습니다. (총 8448개)
✅ 'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_harmful_data_validation'에서 JSON 파일을 읽고 있습니다. (총 1056개)
✅ 'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_normal_data_training'에서 JSON 파일을 읽고 있습니다. (총 28628개)
✅ 'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_normal_data_validation'에서 JSON 파일을 읽고 있습니다. (총 3431개)

데이터를 성공적으로 불러왔습니다.
전체 데이터셋 크기: (41560, 2)

--- 유해성 메일 분류 모델 성능 ---
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      3206
           1       0.98      1.00      0.99       950

    accuracy                           1.00      4156
   macro avg       0.99      

In [12]:
# 모델과 TF-IDF 벡터라이저 저장
joblib.dump(model, 'spam_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

print("✅ 모델과 벡터라이저가 성공적으로 저장되었습니다.")

✅ 모델과 벡터라이저가 성공적으로 저장되었습니다.


In [10]:
import os
import glob
import json

# 말뭉치 데이터가 있는 폴더 경로
folder_path = r'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_normal_data_training'

# 첫 번째 JSON 파일 찾기
first_file = glob.glob(os.path.join(folder_path, "**", "*.json"), recursive=True)[0]

print(f"✅ 첫 번째 파일: {first_file}\n")
print("--- 파일 내용 ---")
try:
    with open(first_file, 'r', encoding='utf-8-sig') as f:
        data = json.load(f)
        # JSON 데이터의 키만 출력
        print("JSON 키:", list(data.keys()))
        # 첫 번째 데이터 항목만 출력
        print("\n--- 첫 번째 데이터 항목 ---")
        if 'data' in data:
            print(data['data'][0])
        else:
            print(data)
except Exception as e:
    print(f"파일을 읽는 중 오류가 발생했습니다: {e}")

✅ 첫 번째 파일: C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_normal_data_training\BU_S0000001.json

--- 파일 내용 ---
JSON 키: ['data']

--- 첫 번째 데이터 항목 ---
{'source_id': 'S0000001', 'title': '제3기(2011~2014) 부천시지역사회복지계획', 'publisher_company': '경기도', 'category_main': '정치', 'category_middle': '지방행정', 'isbn': 'CM00055107', 'collection_name': '사업보고서', 'contents_type': '공개', 'issue_date': '2010', 'corpus': '4. 제3기 지역사회복지계획 수립의 특징 및 방향\n1)주요특징 및 방향\n가. 부천시민의 삶의 질 향상을 위한 복지서비스 강화\n• 저소득층의 기초생활을 보장하고 자활·자립을 위한 기반 조성함\n• 아동과 청소년이 밝고 건강하게 성장하고 글로벌 인재로 육성하기 위한 환경을 조성하고 이를 위한 지원을 강화함\n• 노인과 장애인의 생활안정을 위한 지원을 활성화하고 이들의 사회참여를 증진시킬 수 있는 서비스 강화\n• 자원봉사 문화와 양성평등 문화를 확산하여 ‘더불어 사는 복지도시’를 이룰 수 있는 토대를 마련하도록 함\n나. 복지시설 취약지역에 대한 균형배치로 지역 간 복지서비스의 불균형 해소\n• 기존의 종합사회복지관을 중심으로 노인, 여성, 아동, 청소년 복지시설, 지역자활센터와 연계체계를 활성화하여 복지서비스의 소외지역이 없도록 함\n• 서비스에 대한 홍보를 강화하여 서비스 인지도와 활용도를 높이도록 함\n다. 공공보건의료 확충과 네트워크 구축으로 주민 건강권 확보\n• 보건소의 기능을 예방 중심으로 개편하고 국가적인 보건사업의 변화에 적극적인 대처 및 우리 실정에 맞는 보건사업과 민간 기관과 연계한 사업에 대처할 수

___

## 라이브러리 및 데이터 준비

In [ ]:
import pandas as pd
import glob
import os
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib
import matplotlib.pyplot as plt

# 추가된 라이브러리
import zipfile

# ✅ 수정된 unzip_file 함수
def unzip_file(zip_path, extract_path):
    if os.path.exists(extract_path):
        print(f"✅ 압축 해제된 폴더가 이미 존재합니다: {extract_path}")
        return
        
    print(f"✅ '{os.path.basename(zip_path)}' ZIP 파일 압축을 해제합니다.")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print("✅ 압축 해제 완료!")
    except FileNotFoundError:
        print(f"Error: '{zip_path}' 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"압축 해제 중 오류가 발생했습니다: {e}")

# JSON 파일 불러오기 함수 (기존 코드)
def load_json_data(folder_path, label_as_harmful=True):
    all_files = glob.glob(os.path.join(folder_path, "**", "*.json"), recursive=True)
    if not all_files:
        print(f"경로 '{folder_path}'에서 JSON 파일을 찾을 수 없습니다.")
        return pd.DataFrame()
    
    data_list = []
    print(f"✅ '{folder_path}'에서 JSON 파일을 읽고 있습니다. (총 {len(all_files)}개)")
    
    if 'normal' in folder_path:
        text_key = 'corpus'
    else:
        text_key = 'instruct_text'
        
    for file_path in all_files:
        try:
            with open(file_path, 'r', encoding='utf-8-sig') as f:
                json_data = json.load(f)
                
                if 'data' in json_data and isinstance(json_data['data'], list):
                    for item in json_data['data']:
                        text = item.get(text_key, '')
                        label = 1 if label_as_harmful else 0
                        data_list.append({'text': text, 'label': label})
                elif isinstance(json_data, dict):
                    text = json_data.get(text_key, '')
                    label = 1 if label_as_harmful else 0
                    data_list.append({'text': text, 'label': label})
                else:
                    print(f"Error: {file_path} 파일의 예상치 못한 구조: {type(json_data)}")
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            continue

    df = pd.DataFrame(data_list)
    return df

# 텍스트 전처리 함수 (기존 코드)
def clean_text(text):
    text = str(text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', text)
    text = re.sub(r'[^\w\s\n]|<[^>]+>',' ', text).strip()
    return ' '.join(text.split())

# ✅ ZIP 파일 압축 해제 호출
unzip_file(r'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\유해질의.zip', r'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_harmful_data')
unzip_file(r'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\말뭉치.zip', r'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_normal_data')


# 데이터셋 불러오기 (기존 코드)
harmful_training_data = load_json_data(r'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_harmful_data_training')
harmful_validation_data = load_json_data(r'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_harmful_data_validation')
normal_training_data = load_json_data(r'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_normal_data_training', label_as_harmful=False)
normal_validation_data = load_json_data(r'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_normal_data_validation', label_as_harmful=False)

# 훈련 데이터 합치기 
df_train = pd.concat([harmful_training_data, normal_training_data]).drop_duplicates(subset=['text']).reset_index(drop=True)

# 검증 및 테스트 데이터 합치기 
df_temp = pd.concat([harmful_validation_data, normal_validation_data]).drop_duplicates(subset=['text']).reset_index(drop=True)
df_valid, df_test = train_test_split(df_temp, test_size=0.5, random_state=42, stratify=df_temp['label'])
df_train.reset_index(drop=True, inplace=True)
df_valid.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

print("\n데이터를 성공적으로 불러왔습니다.")
print(f"전체 데이터셋 크기: {df_train.shape}")

# 텍스트 전처리
df_train['text'] = df_train['text'].apply(clean_text)
df_valid['text'] = df_valid['text'].apply(clean_text)
df_test['text'] = df_test['text'].apply(clean_text)

✅ '유해질의.zip' ZIP 파일 압축을 해제합니다.
Error: 'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\유해질의.zip' 파일을 찾을 수 없습니다.
✅ '말뭉치.zip' ZIP 파일 압축을 해제합니다.
Error: 'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\말뭉치.zip' 파일을 찾을 수 없습니다.
✅ 'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_harmful_data_training'에서 JSON 파일을 읽고 있습니다. (총 8448개)
✅ 'C:\Users\admin\Desktop\dev\Python-Practice\project\my_model\extracted_harmful_data_validation'에서 JSON 파일을 읽고 있습니다. (총 1056개)


## TF-IDF 벡터라이저 및 모델 학습

In [ ]:
# TF-IDF 벡터라이저 생성 및 학습
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train = tfidf_vectorizer.fit_transform(df_train['text'])
y_train = df_train['label']

# 로지스틱 회귀 모델 학습
# class_weight='balanced' 옵션으로 데이터 불균형을 해결
model = LogisticRegression(class_weight='balanced', max_iter=1000)
model.fit(X_train, y_train)

# 테스트 데이터 변환
X_test = tfidf_vectorizer.transform(df_test['text'])
y_test = df_test['label']

## 모델 성능 평가

In [ ]:
# 예측
y_pred = model.predict(X_test)

# 분류 보고서
report = classification_report(y_test, y_pred, output_dict=True)
print("--- 유해성 메일 분류 모델 성능 ---")
print(classification_report(y_test, y_pred))

## 모델 성능 시각화

In [ ]:
# 성능 지표를 데이터프레임으로 변환
report_df = pd.DataFrame(report).transpose()

# 0(정상 메일)과 1(스팸 메일) 클래스의 성능 지표만 추출
metrics = report_df.loc[['0', '1'], ['precision', 'recall', 'f1-score']]

# 시각화
metrics.plot(kind='bar', figsize=(10, 6), rot=0)
plt.title('Performance of Spam Classification Model', fontsize=15)
plt.xlabel('Class (0: Normal, 1: Spam)', fontsize=12)
plt.ylabel('Score', fontsize=12)
plt.ylim(0.9, 1.01)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

## 모델 저장

In [ ]:
# 모델과 TF-IDF 벡터라이저 저장
joblib.dump(model, 'spam_model.pkl')
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

print("✅ 모델과 벡터라이저가 성공적으로 저장되었습니다.")